In [0]:
from google.colab import drive
!mkdir /content/data
drive.mount("/content/data")

mkdir: cannot create directory ‘/content/data’: File exists
Drive already mounted at /content/data; to attempt to forcibly remount, call drive.mount("/content/data", force_remount=True).


In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-04-02 08:15:22--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.45.111.123, 35.173.3.255, 52.204.188.97, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.45.111.123|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14977695 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.3’

ngrok-stable-linux- 100%[===================>]  14.28M  9.32MB/s    in 1.5s    

2019-04-02 08:15:24 (9.32 MB/s) - ‘ngrok-stable-linux-amd64.zip.3’ saved [14977695/14977695]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [0]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization, MaxPooling2D,Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import os
import numpy as np
np.random.seed(1337)  # for reproducibility
import pickle
from keras.models import Sequential
# from tensorflow.python.keras import Sequential
from keras.optimizers import SGD, Adam, RMSprop
from keras.callbacks import LearningRateScheduler
from keras.utils import np_utils
K.set_image_data_format('channels_first')
!rm -r Graph
!rm -r GraphODE

rm: cannot remove 'Graph': No such file or directory
rm: cannot remove 'GraphODE': No such file or directory


In [0]:
pip install tensorboardcolab

In [0]:
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
tbc=TensorBoardColab()
# LOG_DIR = './content/data/My Drive/Minor Project/toSend/GraphCNNF95Colab'
# get_ipython().system_raw(
#     'tensorboard --logdir {}  &'
#     .format(LOG_DIR)
# )

# !sudo curl http://localhost:6006
# get_ipython().system_raw('./ngrok http 6006 &')
# ! curl -s http://localhost:4040/api/tunnels | python3 -c \
#     "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Wait for 8 seconds...
TensorBoard link:
https://f6b66f91.ngrok.io


In [0]:
from binary_ops import binary_tanh as binary_tanh_op


H = 1.
kernel_lr_multiplier = 'Glorot'

save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# nn
# batch_size = 50
batch_size = 16
epochs = 50
nb_channel = 1
img_rows = 71 
img_cols = 64
# img_rows = 64 
# img_cols = 64
nb_filters = 32 
nb_conv = 3
nb_pool = 2
nb_hid = 128
nb_classes = 72
use_bias = False

# learning rate schedule
lr_start = 1e-3
lr_end = 1e-4
lr_decay = (lr_end / lr_start)**(1. / epochs)

# BN
epsilon = 1e-6
momentum = 0.9

# dropout
p1 = 0.25
p2 = 0.5
# data needs to be a tuple with two elements.
# the data, shuffled and split between train and test sets
# (X_train, y_train), (X_test, y_test) = load_faces95_data.load_data()
# pickledDataset = open("faces95dataWithPCA.pickle","rb")
pickledDataset = open("/content/data/My Drive/Minor Project/toSend/faces95dataWithImgSalMBD.pickle","rb")
data = pickle.load(pickledDataset)
(X_train, y_train), (X_test, y_test) = data['res']
print("Dataset has been loaded")

X_train = X_train.reshape(1224, 1, 71, 64)
X_test = X_test.reshape(216, 1, 71, 64)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes) * 2 - 1 # -1 or 1 for hinge loss
Y_test = np_utils.to_categorical(y_test, nb_classes) * 2 - 1


Dataset has been loaded
1224 train samples
216 test samples


In [0]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 3),   
                     padding='same', use_bias=use_bias, name='conv4'))
# remove 3 lines below if needed
# model.add(Conv2D(256, kernel_size=(3, 3),   
#                      padding='same', use_bias=use_bias, name='conv4_a'))
# model.add(Conv2D(256, kernel_size=(3, 3),   
#                      padding='same', use_bias=use_bias, name='conv4_b'))
# model.add(Conv2D(256, kernel_size=(3, 3),   
#                      padding='same', use_bias=use_bias, name='conv4_c'))
model.add(MaxPooling2D(pool_size=(2, 2), name='pool4'))
model.add(BatchNormalization(epsilon=epsilon, momentum=momentum, name='bn4'))
model.add(Activation('selu', name='act4'))
model.add(Flatten())
# dense1
model.add(Dense(1024,   use_bias=use_bias, name='dense5'))
model.add(BatchNormalization(epsilon=epsilon, momentum=momentum, name='bn5'))
model.add(Activation('selu', name='act5'))
# dense2
model.add(Dense(nb_classes,   use_bias=use_bias, name='dense6'))
model.add(BatchNormalization(epsilon=epsilon, momentum=momentum, name='bn6'))

opt = Adam(lr=lr_start) 
model.compile(loss='squared_hinge', optimizer=opt, metrics=['acc'])

lr_scheduler = LearningRateScheduler(lambda e: lr_start * lr_decay ** e)

In [0]:
# tbCallBack = keras.callbacks.TensorBoard(log_dir='./content/data/My Drive/Minor Project/toSend/GraphCNNF95Colab', histogram_freq=0, write_graph=True, write_images=True)

history = model.fit(X_train, Y_train,
                    batch_size=batch_size, epochs=epochs,
                    verbose=1, validation_data=(X_test, Y_test),
                    callbacks=[lr_scheduler,TensorBoardColabCallback(tbc)])

model.save('/content/data/My Drive/Minor Project/toSend/saved_models/CNNF95_colab.h5')

score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 1224 samples, validate on 216 samples
Epoch 1/50
1224/1224 [==============================] - 9s 7ms/step - loss: 1.6202 - acc: 0.2132 - val_loss: 1.1649 - val_acc: 0.2778
Epoch 2/50
1224/1224 [==============================] - 8s 7ms/step - loss: 1.2791 - acc: 0.5123 - val_loss: 1.1450 - val_acc: 0.5463
Epoch 3/50
1224/1224 [==============================] - 8s 7ms/step - loss: 1.0887 - acc: 0.6993 - val_loss: 0.8317 - val_acc: 0.6435
Epoch 4/50
1224/1224 [==============================] - 8s 7ms/step - loss: 0.9315 - acc: 0.7876 - val_loss: 0.9789 - val_acc: 0.6620
Epoch 5/50
1224/1224 [==============================] - 8s 7ms/step - loss: 0.8025 - acc: 0.8309 - val_loss: 0.5653 - val_acc: 0.7222
Epoch 6/50
1224/1224 [==============================] - 8s 7ms/step - loss: 0.6816 - acc: 0.8668 - val_loss: 0.6052 - val_acc: 0.7130
Epoch 7/50
1224/1224 [==============================] - 8s 7ms/step - loss: 0.5970 - acc: 0.9069 - val_loss: 0.5833 - val_acc: 0.7546
Epoch 8/50
1224